In [0]:
import pandas as pd

In [0]:
pd_train = pd.read_csv("Dataset/arxiv/train_all.csv",dtype="str")

In [0]:
pd_test = pd.read_csv("Dataset/arxiv/test_all.csv",dtype="str")

In [7]:
pd_train.shape

(200092, 10)

In [8]:
pd_test.shape

(37125, 10)

Clean up abstract using noun only phrase

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
def noun_only_sentence(x_string):
    string_list = x_string.lower().split(".")
    noun_sentence = []
    for my_str in string_list:
        #print(my_str)
        tokens = word_tokenize(my_str)
        #bigrams = ngrams(token,2)
        # remove all tokens that are not alphabetic
        is_noun = lambda pos: pos[:2] == 'NN'
        # do the nlp stuff
        nouns = [word for (word, pos) in nltk.pos_tag(tokens) if is_noun(pos)] 
        noun_sentence.append(nouns)
    return noun_sentence

In [0]:
#abstract_noun = pd_train.abstract.apply(lambda x: noun_only_sentence(x))

In [0]:
#abstract_noun_test = pd_test.abstract.apply(lambda x: noun_only_sentence(x))

In [0]:
import pickle
with open("Dataset/arxiv/abstract_noun_strat.pickle","rb") as file:
  (abstract_noun,abstract_noun_test) = pickle.load(file)

In [0]:
# clean abstract noun so it will contain only alphabetic char

# use english stop_words
stop_words =  list(stopwords.words('english'))

def alphabetic_only_l(x_string):
  list_sent = []    
  for tokens in x_string:
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha() and word not in stop_words and len(word)>3]
    list_sent.append(words)    
  return list_sent

In [0]:
#abs_noun_clean = abstract_noun.apply(lambda x:alphabetic_only_l(x))

In [0]:
#abs_noun_clean_test = abstract_noun_test.apply(lambda x:alphabetic_only_l(x))

In [0]:
import pickle
with open("Dataset/arxiv/abstract_noun_clean_strat.pickle","rb") as file:
  (abs_noun_clean,abs_noun_clean_test) = pickle.load(file)

In [0]:
# features/words chooser using tfidf
def sent_convert(x_string):
  list_sent = []
  for sent in x_string:
    # remove all tokens that are not alphabetic    
    list_sent.append(" ".join(sent))    
  return ". ".join(list_sent)

abs_noun_sent = abs_noun_clean.apply(lambda x:sent_convert(x))

In [0]:
abs_noun_sent_test = abs_noun_clean_test.apply(lambda x:sent_convert(x))

In [13]:
abs_noun_sent.shape

(200092,)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
tf = tf_vectorizer.fit_transform(abs_noun_sent.values)
tf_feature_names = tf_vectorizer.get_feature_names()

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf = tfidf_vectorizer.fit_transform(abs_noun_sent.values)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [15]:
len(tfidf_feature_names)

44819

In [16]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 5.6MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tflearn.data_utils import to_categorical, pad_sequences


Using TensorFlow backend.


In [0]:
tfidf.shape

In [0]:
y_train_label = pd_train.cats.apply(lambda x: x.split("|")[0])

In [19]:
pd_train.shape

(200092, 10)

In [0]:
y_train_label_l = list(y_train_label.unique())

In [0]:
y_train_class = y_train_label.apply(lambda x:y_train_label_l.index(x))

In [0]:
from keras.layers import Input,Dropout
from keras.models import Model


model = Sequential()
# input layer
model.add(Dropout(0.4,input_shape=(tfidf.shape[1],)))
model.add(keras.layers.Dense(5000,activation='relu'))
model.add(Dropout(0.3))

# hidden layer
model.add(keras.layers.Dense(1000,activation='relu'))
model.add(Dropout(0.2))
model.add(keras.layers.Dense(500,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))

# output layer
model.add(keras.layers.Dense(len(y_train_label_l),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_12 (Dropout)         (None, 44819)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 5000)              224100000 
_________________________________________________________________
dropout_13 (Dropout)         (None, 5000)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 1000)              5001000   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_25 (Dense)             (None, 100)               50100     
__________

In [0]:
y_test_class = pd_test.cats.apply(lambda x: x.split("|")[0]).apply(lambda x:y_train_label_l.index(x))

In [0]:
tfidf_test = tfidf_vectorizer.transform(abs_noun_sent_test.values)

In [0]:
y_test_one_hot = to_categorical(y_test_class.values,20)

In [38]:
from sklearn.model_selection import train_test_split, StratifiedKFold

import numpy as np
mini_batch = 20000
total_docs = tfidf.shape[0]
total_batch = int(tfidf.shape[0]/mini_batch)+1
#range_docs = range(0,total_docs)
#range_docs = train


for it in range(3):
  k_sample = StratifiedKFold(n_splits=total_batch, shuffle=True, random_state=1).split(range(len(y_train_class.values)), y_train_class.values)
  for batch in k_sample:
      # random sample dataset
      #r_index = random.sample(list(range_docs),mini_batch)
      #r_index_start = mini_batch*batch
      #r_index_end = mini_batch*(batch+1) if batch<total_batch-1 else total_docs
      X_batch = tfidf[batch[1]]
      #y_batch = dataset.iloc[r_index].subjects.apply(lambda x:labeling_subjects(x))
      #y_batch = np.array([])
      #for x in dataset.iloc[r_index].subjects:
      #    y_batch = np.append(y_batch,labeling_subjects(x),axis=0)
      #y_batch
      #y_batch = y_batch.reshape(len(r_index),len(label_class))
      y_batch = to_categorical(y_train_class[batch[1]].values,20)

      model.fit(X_batch,y_batch,epochs=1,batch_size=200)
  y_pred = model.predict_classes(tfidf_test)
  from sklearn.metrics import classification_report,accuracy_score,cohen_kappa_score
  acc = accuracy_score(y_test_class,y_pred)
  kap = cohen_kappa_score(y_test_class,y_pred)
  print("accuracy:",acc)
  print("kappa:",kap)
  print(classification_report(y_test_class,y_pred))
  model.save("Dataset/arxiv/nn-model-drop-{}.h5".format(it))

Epoch 1/1
18195/18195 [==============================] - 45s 2ms/step - loss: 1.5103 - acc: 0.5546
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.9797 - acc: 0.7083
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.8677 - acc: 0.7324
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.8341 - acc: 0.7441
Epoch 1/1
18193/18193 [==============================] - 35s 2ms/step - loss: 0.7880 - acc: 0.7548
Epoch 1/1
18191/18191 [==============================] - 35s 2ms/step - loss: 0.7829 - acc: 0.7573
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.7527 - acc: 0.7644
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.7408 - acc: 0.7644
Epoch 1/1
18188/18188 [==============================] - 35s 2ms/step - loss: 0.7350 - acc: 0.7692
Epoch 1/1
18182/18182 [==============================] - 35s 2ms/step - loss: 0.7277 - acc: 0.7728
Epoch 1/1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1/1
18195/18195 [==============================] - 36s 2ms/step - loss: 0.6776 - acc: 0.7851
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.6438 - acc: 0.7904
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.6202 - acc: 0.7988
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.6215 - acc: 0.7994
Epoch 1/1
18193/18193 [==============================] - 35s 2ms/step - loss: 0.6014 - acc: 0.8035
Epoch 1/1
18191/18191 [==============================] - 35s 2ms/step - loss: 0.6028 - acc: 0.8045
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.5882 - acc: 0.8053
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.5848 - acc: 0.8074
Epoch 1/1
18188/18188 [==============================] - 35s 2ms/step - loss: 0.5896 - acc: 0.8050
Epoch 1/1
18182/18182 [==============================] - 35s 2ms/step - loss: 0.5765 - acc: 0.8122
Epoch 1/1


In [39]:
for it in range(5):
  k_sample = StratifiedKFold(n_splits=total_batch, shuffle=True, random_state=1).split(range(len(y_train_class.values)), y_train_class.values)
  for batch in k_sample:
      # random sample dataset
      #r_index = random.sample(list(range_docs),mini_batch)
      #r_index_start = mini_batch*batch
      #r_index_end = mini_batch*(batch+1) if batch<total_batch-1 else total_docs
      X_batch = tfidf[batch[1]]
      #y_batch = dataset.iloc[r_index].subjects.apply(lambda x:labeling_subjects(x))
      #y_batch = np.array([])
      #for x in dataset.iloc[r_index].subjects:
      #    y_batch = np.append(y_batch,labeling_subjects(x),axis=0)
      #y_batch
      #y_batch = y_batch.reshape(len(r_index),len(label_class))
      y_batch = to_categorical(y_train_class[batch[1]].values,20)

      model.fit(X_batch,y_batch,epochs=1,batch_size=200)
  y_pred = model.predict_classes(tfidf_test)
  from sklearn.metrics import classification_report,accuracy_score,cohen_kappa_score
  acc = accuracy_score(y_test_class,y_pred)
  kap = cohen_kappa_score(y_test_class,y_pred)
  print("accuracy:",acc)
  print("kappa:",kap)
  print(classification_report(y_test_class,y_pred))
  model.save("Dataset/arxiv/nn-model-drop-b{}.h5".format(it))

Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.4725 - acc: 0.8408
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.4559 - acc: 0.8486
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.4464 - acc: 0.8521
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.4330 - acc: 0.8520
Epoch 1/1
18193/18193 [==============================] - 35s 2ms/step - loss: 0.4357 - acc: 0.8503
Epoch 1/1
18191/18191 [==============================] - 35s 2ms/step - loss: 0.4365 - acc: 0.8506
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.4198 - acc: 0.8583
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.4209 - acc: 0.8578
Epoch 1/1
18188/18188 [==============================] - 35s 2ms/step - loss: 0.4144 - acc: 0.8595
Epoch 1/1
18182/18182 [==============================] - 35s 2ms/step - loss: 0.4086 - acc: 0.8608
Epoch 1/1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1/1
18195/18195 [==============================] - 36s 2ms/step - loss: 0.3902 - acc: 0.8701
Epoch 1/1
18195/18195 [==============================] - 36s 2ms/step - loss: 0.3767 - acc: 0.8730
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.3711 - acc: 0.8754
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.3694 - acc: 0.8757
Epoch 1/1
18193/18193 [==============================] - 35s 2ms/step - loss: 0.3563 - acc: 0.8791
Epoch 1/1
18191/18191 [==============================] - 35s 2ms/step - loss: 0.3564 - acc: 0.8823
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.3400 - acc: 0.8842
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.3454 - acc: 0.8846
Epoch 1/1
18188/18188 [==============================] - 35s 2ms/step - loss: 0.3457 - acc: 0.8839
Epoch 1/1
18182/18182 [==============================] - 35s 2ms/step - loss: 0.3403 - acc: 0.8877
Epoch 1/1


In [40]:
for it in range(5):
  k_sample = StratifiedKFold(n_splits=total_batch, shuffle=True, random_state=1).split(range(len(y_train_class.values)), y_train_class.values)
  for batch in k_sample:
      # random sample dataset
      #r_index = random.sample(list(range_docs),mini_batch)
      #r_index_start = mini_batch*batch
      #r_index_end = mini_batch*(batch+1) if batch<total_batch-1 else total_docs
      X_batch = tfidf[batch[1]]
      #y_batch = dataset.iloc[r_index].subjects.apply(lambda x:labeling_subjects(x))
      #y_batch = np.array([])
      #for x in dataset.iloc[r_index].subjects:
      #    y_batch = np.append(y_batch,labeling_subjects(x),axis=0)
      #y_batch
      #y_batch = y_batch.reshape(len(r_index),len(label_class))
      y_batch = to_categorical(y_train_class[batch[1]].values,20)

      model.fit(X_batch,y_batch,epochs=1,batch_size=200)
  y_pred = model.predict_classes(tfidf_test)
  from sklearn.metrics import classification_report,accuracy_score,cohen_kappa_score
  acc = accuracy_score(y_test_class,y_pred)
  kap = cohen_kappa_score(y_test_class,y_pred)
  print("accuracy:",acc)
  print("kappa:",kap)
  print(classification_report(y_test_class,y_pred))
  model.save("Dataset/arxiv/nn-model-drop-c{}.h5".format(it))

Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.2207 - acc: 0.9260
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.1968 - acc: 0.9345
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.2114 - acc: 0.9305
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.2155 - acc: 0.9300
Epoch 1/1
18193/18193 [==============================] - 35s 2ms/step - loss: 0.2033 - acc: 0.9321
Epoch 1/1
18191/18191 [==============================] - 35s 2ms/step - loss: 0.2026 - acc: 0.9319
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.2027 - acc: 0.9330
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.1995 - acc: 0.9338
Epoch 1/1
18188/18188 [==============================] - 35s 2ms/step - loss: 0.1991 - acc: 0.9330
Epoch 1/1
18182/18182 [==============================] - 35s 2ms/step - loss: 0.1979 - acc: 0.9340
Epoch 1/1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1/1
18195/18195 [==============================] - 36s 2ms/step - loss: 0.1944 - acc: 0.9370
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.1970 - acc: 0.9347
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.1979 - acc: 0.9336
Epoch 1/1
18195/18195 [==============================] - 35s 2ms/step - loss: 0.1872 - acc: 0.9389
Epoch 1/1
18193/18193 [==============================] - 35s 2ms/step - loss: 0.1957 - acc: 0.9343
Epoch 1/1
18191/18191 [==============================] - 35s 2ms/step - loss: 0.1892 - acc: 0.9380
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.1825 - acc: 0.9413
Epoch 1/1
18190/18190 [==============================] - 35s 2ms/step - loss: 0.1870 - acc: 0.9373
Epoch 1/1
18188/18188 [==============================] - 35s 2ms/step - loss: 0.1864 - acc: 0.9366
Epoch 1/1
18182/18182 [==============================] - 35s 2ms/step - loss: 0.1874 - acc: 0.9388
Epoch 1/1


In [0]:
y_pred = model.predict_classes(tfidf_test)

In [0]:
y_pred

In [0]:
from sklearn.metrics import classification_report,accuracy_score,cohen_kappa_score
acc = accuracy_score(y_test_class,y_pred)
kap = cohen_kappa_score(y_test_class,y_pred)
print("accuracy:",acc)
print("kappa:",kap)
print(classification_report(y_test_class,y_pred))